In [11]:
# 22.06.23 Bg Ausgabe mit +-Zeichen 
# 22.06.23 BG tradeDate + und dateTime als echte Datums/Uhrzeitfelder 
# 21.06.23 BG ibCommission - Feld ergänzt  
# 21.06.23 BG neue Feldnamen in das Program integriert (war alles OK) 
# 17.06.23 BG openClose in Langtext umbenannt 
# 15.06.23 BG assetCategory eergänzt (OPT) , multiplier
# 12.06.23 BG erster Wurf
#
# 
#
#verwendeter Report: Trades
#
# Infos zu dem Report 
# https://ibkrguides.com/reportingreference/reportguide/tradesfq.htm
#
import locale
from xml.dom.minidom import parse
from datetime import datetime

import pyodbc  # Datenbankanbindung an Access 

locale.setlocale(locale.LC_ALL, 'de_DE')
# locale.setlocale(locale.decimal_point, ',' )
#print (locale.localeconv())

def IBStr2Date ( dateStr):
    formatting = "%Y%m%d"
    date = datetime.strptime(dateStr, formatting)
    return date

def IBStr2DateTime ( dateTimeStr):
    formatting = "%Y%m%d;%H%M%S"
    dateTime = datetime.strptime(dateTimeStr, formatting)
    return dateTime


def insertTrades ( item , dbCursor , dbConn):
    
    description = item.getAttribute ("activityDescription") 
    code = item.getAttribute ("activityCode") 
    transactionID = item.getAttribute ("transactionID") 
    tradeID = item.getAttribute ("tradeID")  
    if ( tradeID == ""):
        tradeID = 0 

    accountID = item.getAttribute ("accountId") 
        
    
    #dateTimeStr = item.getAttribute ("dateTime") # Nur in dem Feld ist immer ein Datum drin  
    #dateTime = IBStr2DateTime ( dateTimeStr )
    
    # Weitere Felder bei Trades 
    conid = item.getAttribute ("conid") 
    description = item.getAttribute ("description") 
    dax = description.split(' ', 1)[0]
    symbol = item.getAttribute ("symbol")   
    symbol = symbol.split(' ', 1)[0]

    # Erklärung: 
        # Beim Kaufen ist Cost die Gesamtkosten der Anschaffung 
        # beim Verkauf sind Cost die Anteiligen Anschaffungskosten 
        # und der  fifoPnlRealized ist der Gewinn/Verlust, der durch diesen Verkauf realiert wird

    assetCategory=item.getAttribute ("assetCategory") # 15.06 Neu dazu 
    multiplier = int(item.getAttribute ("multiplier")) # 15.06 Neu dazu  
    transactionType = item.getAttribute ("transactionType")# 15.06 Neu dazu
    fifoPnlRealized = float(item.getAttribute ("fifoPnlRealized")) # 15.06. Beu - 

    if ( dax == "DAX" ):
        symbol = dax 

    quantity = item.getAttribute ("quantity")  
    cost = round(float(item.getAttribute ("cost")),2)      
    putCall = item.getAttribute ("putCall") 

    openClose= item.getAttribute ("openCloseIndicator") 
    if openClose == "C" :
        openClose = "Close"
    elif openClose == "O":
        openClose = "Open"
    
    strike = item.getAttribute ("strike") 

    tradePrice = round(float(item.getAttribute ("tradePrice")),2) 
    result = float(item.getAttribute ("fifoPnlRealized")) 
    buySell = item.getAttribute ("buySell") 
    quantity = item.getAttribute ("quantity") 
    # Buy or sell.
    buySell = item.getAttribute ("buySell")
    IBCurrency = item.getAttribute ("currency") # 22.06. Name geändert 
    
    expiry = item.getAttribute ("expiry") 
    if ( expiry != "" ):
        # 22.06 UMgestellt auf Funktion
        expiryDate = IBStr2Date ( expiry)       
    else:
        expiryDate = "01.01.1900"

    tradeDateStr = item.getAttribute ("tradeDate") 
    # 22.06 BG umgestellt auf Function 
    tradeDate = IBStr2Date(tradeDateStr)

    dateTimeStr = item.getAttribute ("dateTime") # 22.06. Jetzt original Datumsfeld 
    # Abweichender Name, weil DateTime ein Schlüsselwort ist 
    IBDateTime = IBStr2DateTime ( dateTimeStr )


    ibCommission = item.getAttribute ("ibCommission")
    if ( ibCommission == "" ):
        ibCommission = 0.00
    else:
         ibCommission = float(ibCommission)
 
    # print ( "Transaktion:" , transactionID, " COnID", conid, "Beschreibung " , description , "Datum: " , tradeDatum , "Experiy", expiryDatum , "Time: ", dateTime) 
    # In die Datenbank eintragen 
    # print ( assetCategory, "-", multiplier, "-", transactionType, "-", fifoPnlRealized )

    if ( transactionID != ""):
        transactionID = int(transactionID )

        sql = "INSERT INTO trades "
        sql += "(transactionID, accountID, tradeID, description, tradeDate, IBDateTime " # IBDateTime,
        sql += ", putCall, buySell, cost, strike, expiryDate, conid, tradePrice, openclose, quantity, IBCurrency, symbol "
        sql += ", assetCategory, multiplier, transactionType, fifoPnlRealized , ibCommission "
        sql += ") "
        sql += "VALUES (?, ?, ?, ?, ?, ? "      
        sql += ", ?, ?, ?, ? , ?, ?, ?, ?, ?, ?, ? "
        sql += ", ?, ?, ?, ?, ? "
        sql += ")"
    
        # print (sql)

        data = (transactionID, accountID, tradeID, description , tradeDate, IBDateTime \
               , putCall, buySell, cost, strike, expiryDate, conid, tradePrice, openClose, quantity, IBCurrency, symbol \
                , assetCategory, multiplier, transactionType, fifoPnlRealized, ibCommission \
            )
    
        dbCursor.execute(sql, data)
        dbConn.commit()


# --------- Start des Programms --------------------------

#Datenbank öffnen 
# Verbindung zur Access-Datenbank herstellen
dbConn = pyodbc.connect(r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=.\ib.accdb;')

# Cursor-Objekt erstellen
dbCursor = dbConn.cursor()

#alle Daten löschen 
sql= "DELETE FROM trades"
dbCursor.execute (sql) 

#domobject = parse('trades-complett.xml')

dateiname = "ib-info-trades.xml"
domobject = parse (dateiname)
elementname = 'Trade'

i = 0 ; 
limit = 70 
for e in domobject.getElementsByTagName( elementname ):
    #Übertragen der Daten in die Datenbank 
    insertTrades ( e , dbCursor, dbConn)
    
    i = i+1
    if i % limit == 0  :
        print ("")
        print (i, end="")
    print("+", end="")

print ("")
print ( "Anzahl Datensätze eingetragen :" , i ) 
    
# Änderungen in der Datenbank speichern
dbConn.commit()
# Verbindung schließen
dbConn.close()


    


+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
70++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
140++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
210++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
280++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
350++++++++++++++++++++++++++++++++++++++++++++++++++++++
Anzahl Datensätze eingetragen : 403
